Код ниже позволяет, используя заранее полученный токен авторизации (ключ доступа) к VK API, парсить посты из глобального поиска по одному или нескольким ключевым словам. В данном случае задействованы методы newsfeed.search и wall.getComments. Была предпринята попытка обойти ограничение в 1000 результатов с использованием сдвига по временным отметкам = код создает запрос к серверу с новой временной отметкой каждый раз, когда общее число выгруженных постов достигает лимита.

In [1]:
import requests
import math
import pandas as pd
import datetime as dt
from tqdm import tqdm_notebook
from time import sleep

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# прочитать токен авторизации (ключ доступа) из файла ``access_token.txt``

with open('access_token.txt', mode = 'r', encoding = 'UTF-8') as infile:
    token = infile.read()

In [4]:
# определить функцию ``vk_downlooad``, которая формирует http-запрос к серверу указанием метода VK API и его параметров 

def vk_download(method, params):
    url = 'https://api.vk.com/method/' + method + '?' + params + '&access_token=' + token + '&v=' + '5.131'
    response = requests.get(url)
    return response.json()

In [5]:
# определить функции ``unix`` и ``unix_to_str``, которые переводят временную отметку (timestamp) в формат строки и обратно 

def unix(datetime_str):
    datetime = dt.datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
    return int(datetime.timestamp())

def unix_to_str(unix):
    return str(dt.datetime.fromtimestamp(unix))

In [6]:
# определить функции ``get_info`` и ``del_duplicates`` для предобработки массива данных

def get_info(full_lst):
    L = []
    for post in full_lst:
        try:
            L.append({'owner_id': post['owner_id'],
                      'post_id': post['id'], 
                      'post_text': post['text'].replace('\n', ''), 
                      'comments': post['comments']['count'], 
                      'date': unix_to_str(post['date'])[0:10]})
        except KeyError:
            pass
    return L

def del_duplicates(full_lst):
    id_lst = [post['post_id'] for post in full_lst]
    for post in full_lst:
        if id_lst.count(post['post_id']) > 1:
            full_lst.remove(post)
    return full_lst

In [7]:
def create_lst(full_lst):
    final_lst = get_info(full_lst)  # вытащить информацию об id_пользователя, id_поста, тексте поста, числе комментариев 
                                    # к нему и временной отметке
    return final_lst

In [8]:
# определить функцию ``get_tag``, которая выгружает все посты из глобального поиска по заданному ключевому слову 
# (= встречается в тексте 1 раз и более)

def get_tag(tag, start_time, end_time):
    fulltag_1 = []
    dates = ['2022-05-04 12:00:00']
    start_from = 0
    start_time = unix(start_time)
    end_time = unix(end_time)
    for i in tqdm_notebook(range(10000)):
        n1 = unix_to_str(start_time).split()[0].split('-')[2]
        n2 = unix_to_str(start_time).split()[1].split(':')[0]
        if dates[-1].split()[0].split('-')[2] == n1 and dates[-1].split()[1].split(':')[0] == n2:
            break
        info = vk_download(method = 'newsfeed.search', params = f'q={tag}&count=200&start_from={start_from}&start_time={start_time}&end_time={end_time}')
        fulltag_1.extend(info['response']['items'])
        dates.extend([unix_to_str(post['date']) for post in info['response']['items']])
        sleep(0.4)
        try:
            start_from = info['response']['next_from']
        except KeyError:
            start_from = 0
            end_time = unix(dates[-1])
        print(dates[-1])
    return create_lst(fulltag_1)

In [9]:
# определить функцию ``get_taglst``, которая выгружает все посты из глобального поиска по СПИСКУ* ключевых слов

def get_tag_lst(tag_lst, start_time, end_time):
    info = []
    for tag in tag_lst:
        L = get_tag(tag = tag, start_time = start_time, end_time = end_time)
        info.extend(L)
    return del_duplicates(info)

In [10]:
# определить функцию ``get_comments``, которая выгружает все комментарии к собранным постам

def get_comments(sample):
    sample = sample.set_index('post_id').T.to_dict()
    comments = []
    for k, v in tqdm_notebook(sample.items()):
        N = math.ceil(v['comments'] / 100)
        for i in range(N):
            sleep(0.4)
            info = vk_download(method = 'wall.getComments', 
                               params = f'owner_id={v["owner_id"]}&post_id={k}&count=100&offset={i*100}')
            try:
                comments.extend(info['response']['items'])
            except:
                pass
    for comment in tqdm_notebook(comments):
        try:
            if comment['thread']['count'] >= 1 and 'deleted' not in comment.keys():
                n = math.ceil(comment['thread']['count'] / 100)
                owner_id = comment['owner_id']
                post_id = comment['post_id']
                comment_id = comment['id']
                for i in range(n):
                    sleep(0.4)
                    info = vk_download(method = 'wall.getComments', 
                                       params = f'owner_id={owner_id}&post_id={post_id}&comment_id={comment_id}&count=100&offset={i*100}')
                    comments.extend(info['response']['items'])
            else:
                pass
        except:
            pass
    comments_final = []
    for k, v in sample.items():
        for comment in tqdm_notebook(comments):
            try:
                if comment['post_id'] == k:
                    comments_final.append({'owner_id': v['owner_id'], 
                                           'post_text': v['post_text'].replace('\n', ''), 
                                           'comment_text': comment['text']})
            except:
                pass
    return comments_final

*Попробуем выгрузить из глобального поиска все посты с упоминанием ключевых слов ``безработица`` и ``банк`` за временной промежуток в полчаса: с 3 июля 2023 года 12 часов дня до 3 июля 2023 года 12 часов дня и 30 минут.*

In [11]:
# определим список ключевых слов в переменную ``tag_lst``

tag_lst = ['безработица', 'банк']

In [12]:
# выгрузим все посты в переменную ``posts`` с помощью функции ``get_tag_lst``

posts = get_tag_lst(tag_lst = tag_lst, 
                    start_time = '2023-07-03 12:00:00', 
                    end_time = '2023-07-03 12:30:00')

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-07-03 12:00:01


  0%|          | 0/10000 [00:00<?, ?it/s]

2023-07-03 12:21:23


In [13]:
# узнаем количество постов
len(posts)

213

In [14]:
posts = pd.DataFrame(posts)

In [15]:
posts

,owner_id,post_id,post_text,comments,date
0,-92331941,135303,#конференция #лайкооткликаемо #анонимно #фэндо...,0,2023-07-03
1,-203811109,258,Уровень безработицы по методологии Международн...,0,2023-07-03
2,693135598,600,Уровень безработицы по методологии Международн...,0,2023-07-03
3,-59605730,322686,Дорогие жители города Тотьма. Хочу поделиться ...,4,2023-07-03
4,-217664303,393,📊Ситуация на рынке труда в муниципальном район...,0,2023-07-03
...,...,...,...,...,...
208,-217862702,1510,Решайте любые жизненные ситуации с bincredit ...,0,2023-07-03
209,-207558833,29300,Закаточная машинка Кредмаш МЗП 1-1 с полуавтом...,0,2023-07-03
210,-176228802,37910,"Это чаще встречается, нежели Вам кажется….Сего...",0,2023-07-03
211,-196808207,49360,"Соседи, полезное. Теперь, сдавая банки и бутыл...",0,2023-07-03
